# 데이터 확장

기존에 전라 방언에 적용했던 처리를 확장하여 다른 4개 방언에도 적용

In [1]:
import pandas as pd
import numpy as np
import pathlib
import random
import re
import seaborn as sns

RANDOM_SEED = 19439
random.seed(RANDOM_SEED)

## 강원 방언

In [2]:
df = pd.read_csv("../datas/output/gangwon_dialect_train_age.csv", index_col=0)
df

,방언,표준어,연령대,출처 파일
0,글쎄요,글쎄요,50대,DGDQ20000020.json
1,예?,예?,60대 이상,DGDQ20000020.json
2,뭔 얘기부터 할까유?,뭔 얘기부터 할까요?,60대 이상,DGDQ20000020.json
3,바다 얘기 좀 해주세요 {laughing},바다 얘기 좀 해주세요 {laughing},50대,DGDQ20000020.json
4,바다에요?,바다에요?,60대 이상,DGDQ20000020.json
...,...,...,...,...
1573232,식당에 삐집고 들어갔나?,식당에 비집고 들어갔나?,20대,DGTS20004276.json
1573233,음식점에 삐집고 들어갔나?,음식점에 비집고 들어갔나?,20대,DGTS20004276.json
1573234,쉰 음석이네요.,쉰 음식이네요.,20대,DGTS20004276.json
1573235,이 음석은 쉬었어요.,이 음식은 쉬었어요.,20대,DGTS20004276.json


In [ ]:
# 테스트 데이터로 동일한 과정을 진행하고 싶을 때
df = pd.read_csv("../datas/output/gangwon_dialect_test_age.csv", index_col=0)
df

### 데이터 전처리

In [3]:
df["방언"] = df["방언"].str.replace(r"\(\(\)\)", '', regex=True)
df["표준어"] = df["표준어"].str.replace(r"\(\(\)\)", '', regex=True)

df[df["방언"].str.contains("\(\(\)\)")|df["표준어"].str.contains("\(\(\)\)")]

,방언,표준어,연령대,출처 파일


In [4]:
phrase_words = df["방언"].str.extract(r"\(\((.*)\)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "방언"]
    sentence = re.sub(r"\(\(.*\)\)", phrase_words[i], sentence)
    df.loc[i, "방언"] = sentence

phrase_words = df["표준어"].str.extract(r"\(\((.*)\)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "표준어"]
    sentence = re.sub(r"\(\(.*\)\)", phrase_words[i], sentence)
    df.loc[i, "표준어"] = sentence

df.loc[phrase_words.index]

,방언,표준어,연령대,출처 파일
1192,까 여기까지가 이제 한줄로 요약하면,까 여기까지가 이제 한줄로 요약하면,20대,DGDQ20000032.json
3473,아 요즘에 가방 속 먼지를 청소해주는 더스트 부 뭐 막 이런 게 있대.,아 요즘에 가방 속 먼지를 청소해주는 더스트 부 뭐 막 이런 게 있대.,20대,DGDQ20000053.json
3580,그루일 야자 할 때마다 졸잖아,그루일 야자 할 때마다 졸잖아,20대,DGDQ20000061.json
3698,옵 다 배드민턴만 치더라?,옵 다 배드민턴만 치더라?,20대,DGDQ20000061.json
4643,제가 내년 선택 과목에 인제 탐구 과목 세 개를 고르는데,제가 내년 선택 과목에 인제 탐구 과목 세 개를 고르는데,10대,DGDQ20000067.json
...,...,...,...,...
1057576,다 고년 때민이지.,다 고년 때문이지.,50대,DGIN21920661.json
1057732,그 년이네 니 웬수.,그 년이네 네 웬수.,50대,DGIN21920662.json
1057745,저년 아니 쟤 때미네,저년 아니 쟤 때문에,50대,DGIN21920662.json
1058137,츠녀가 꼬 택시 안에서 잠이 들 정도로 술을 마시고 다녀?,처녀가 꼬 택시 안에서 잠이 들 정도로 술을 마시고 다녀?,50대,DGIN21920664.json


In [5]:
phrase_words = df["방언"].str.extract(r"\((.*)\)\/\(.*\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "방언"]
    sentence = re.sub(r"\(.*\)\/\(.*\)", phrase_words[i], sentence)
    df.loc[i, "방언"] = sentence

phrase_words = df["표준어"].str.extract(r"\(.*\)\/\((.*)\)", expand=False).dropna()
for i in phrase_words.index:
    sentence = df.loc[i, "표준어"]
    sentence = re.sub(r"\(.*\)\/\(.*\)", phrase_words[i], sentence)
    df.loc[i, "표준어"] = sentence

df.loc[phrase_words.index]

,방언,표준어,연령대,출처 파일
9583,답장을 근데 뭐 이 학기 돼가주구 우리가 그냥 계속 연락하다 보니까,답장을 근데 뭐 이 학기 돼돼가지고 우리가 그냥 계속 연락하다 보니까,20대,DGDQ20000090.json
11452,엄마는 아이를 놀이기구에태꿨다.,엄마는 아이를 놀이기구에태웠다.,50대,DGDQ20000103.json
11517,알면서도 모른척하문서 시치미를 뗐다.,알면서도 모른척하면서 시치미를 뗐다.,50대,DGDQ20000103.json
11584,나는 간장에 절인 마늘장쩨이를 즐겨 먹는다.,나는 간장에 절인 마늘짱아찌를 즐겨 먹는다.,50대,DGDQ20000103.json
11586,이 집은 더덕구이와 감장쩨이로 유명하다.,이 집은 더덕구이와 감짱아찌로 유명하다.,50대,DGDQ20000103.json
...,...,...,...,...
1148029,먹고 살기가 힘들어 몇 시간이 되었어도 마수거리도 못해어.,먹고 살기가 힘들어 몇 시간이 되었어도 마수걸이 못했어.,60대 이상,DGTS20000149.json
1388300,아이 괜찮아요.거낭 구경하고 있아요.,아니 괜찮아요.그냥 구경하고 있어요.,40대,DGTS20000601.json
1388409,아이 멋있다고 생각했어 .하물미 치약을 밑에서부터 짜는 거 부터해서.,아니 멋있다고 생각했어 .하물며 치약을 밑에서부터 짜는 거 부터해서.,40대,DGTS20000601.json
1487513,감이 바로 내영감아이와.,감이 바로 내아니오.,40대,DGTS20000941.json


In [6]:
laugh_index = df["방언"].str.contains("\{laughing\}")

df["방언"] = df["방언"].str.replace(r"\{laughing\}", "", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\{laughing\}", "", regex=True)
df[laugh_index]

,방언,표준어,연령대,출처 파일
3,바다 얘기 좀 해주세요,바다 얘기 좀 해주세요,50대,DGDQ20000020.json
51,그건 맞어 .,그건 맞어 .,50대,DGDQ20000020.json
115,김치도 진딱 해 가지고 여기저기 퍼주니,김치도 #진딱 해 가지고 여기저기 퍼주니,60대 이상,DGDQ20000020.json
183,강원도 사투리를 내가 많이 한대.,강원도 사투리를 내가 많이 한대.,60대 이상,DGDQ20000020.json
1630,기독교 아닌 사람들이 더 많아.,기독교 아닌 사람들이 더 많아.,30대,DGDQ20000034.json
...,...,...,...,...
1051062,.,.,40대,DGIN21920532.json
1051649,.,.,60대 이상,DGIN21920534.json
1051698,&name9&씨가 그렇게 말해주이시이까 열심히 조사해서 온 보람이 있는데요?,&name9&씨가 그렇게 말해주시니까 열심히 조사해서 온 보람이 있는데요?,60대 이상,DGIN21920534.json
1052437,,,60대 이상,DGIN21920634.json


In [7]:
stutter_sentence = df["방언"].str.contains(r"\-.*\-", regex=True)
df["방언"] = df["방언"].str.replace(r"\-.*\-", "", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\-.*\-", "", regex=True)

df.loc[stutter_sentence]

,방언,표준어,연령대,출처 파일
17,근데 그 티비에 나오는데 보니까,근데 그 티비에 나오는데 보니까,60대 이상,DGDQ20000020.json
22,지금도 누가 울어도 하고 저 명파 그 사람 죽은 사람 그 누구야?,지금도 누가 울어도 하고 저 명파 그 사람 죽은 사람 그 누구야?,60대 이상,DGDQ20000020.json
26,어떠한 자신감도 있겠지만,어떠한 자신감도 있겠지만,60대 이상,DGDQ20000020.json
32,형님 말 빨라.,형님 말 빨라.,50대,DGDQ20000020.json
46,알았다고 고개 끄덕끄덕 대면서 그렇게 해서 내가 목소리가 큰 줄 알았다니까?,알았다고 고개 끄덕끄덕 대면서 그렇게 해서 내가 목소리가 큰 줄 알았다니까?,60대 이상,DGDQ20000020.json
...,...,...,...,...
1111025,국민들은 두 당 간의 전쟁에 아쭈 무관심한 태도를 보였어.,국민들은 두 당 간의 전쟁에 아주 무관심한 태도를 보였어.,50대,DGTS20000089.json
1111533,그는 항상 지 장모가 좀 짜증스럽다고 그택 생각했어. 한편으로는 어머니와 딸의 관...,그는 항상 자기 장모가 좀 짜증스럽다고 늘 생각했어. 한편으로는 어머니와 딸의 관...,30대,DGTS20000091.json
1144408,가난한 처녀는 지 누디기 드레스를 부끄러워했어.,가난한 처녀는 자기 누더기 드레스를 부끄러워했어.,60대 이상,DGTS20000144.json
1147210,누 빨간 옷 입은 긴 까만 머리?,누구 빨간 옷 입은 긴 까만 머리?,40대,DGTS20000148.json


### 대체어 처리

In [8]:
대체어_위치_방언 = df["방언"].str.contains(r"\&\S+[0-9]\&", regex=True)
대체어_위치_표준어 = df["표준어"].str.contains(r"\&\S+[0-9]\&", regex=True)

In [9]:
df["방언"] = df["방언"].str.replace(r"\&name[0-9]+\&", "name", regex=True)
df["방언"] = df["방언"].str.replace(r"\&address[0-9]+\&", "address", regex=True)
df["방언"] = df["방언"].str.replace(r"\&company-name[0-9]+\&", "companyname", regex=True)
df["방언"] = df["방언"].str.replace(r"\&companyname[0-9]+\&", "companyname", regex=True)

df[대체어_위치_방언]

,방언,표준어,연령대,출처 파일
69,아니 내가 제일 젊진 않고 여기 저 저 name이 엄마 name이 엄마가 더 젊지.,아니 내가 제일 젊진 않고 여기 저 저 &name1&이 엄마 &name1&이 엄마가...,60대 이상,DGDQ20000020.json
70,name이 엄마가 나보다 한 살 아래지 나이로 따지면,&name1&이 엄마가 나보다 한 살 아래지 나이로 따지면,60대 이상,DGDQ20000020.json
288,강릉시 주문진읍에 사는 스무살 name이라고 합니다.,강릉시 주문진읍에 사는 스무살 &name1&이라고 합니다.,20대,DGDQ20000027.json
487,저희가 name 선생님이라는 캡틴을 잘 만나 가지고,저희가 &name1& 선생님이라는 캡틴을 잘 만나 가지고,20대,DGDQ20000027.json
609,name이라는 애가 있습니다.,&name2&이라는 애가 있습니다.,20대,DGDQ20000027.json
...,...,...,...,...
1063038,그러다 name가 저보고 너 다 들었지?,그러다 &name3&가 저보고 너 다 들었지?,50대,DGIN21920687.json
1063044,name보다 행복한 아인 거 같아요.,&name3&보다 행복한 아인 거 같아요.,50대,DGIN21920687.json
1063045,name는 엄마랑 아빠가 다같이 못산다 하는데,&name3&는 엄마랑 아빠가 다같이 못산다 하는데,50대,DGIN21920687.json
1063047,그래 사램은 말이다 name아.,그래 사람은 말이다 &name4&아.,50대,DGIN21920687.json


In [10]:
df["표준어"] = df["표준어"].str.replace(r"\&name[0-9]\&", "name", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\&address[0-9]\&", "address", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\&company-name[0-9]\&", "companyname", regex=True)
df["표준어"] = df["표준어"].str.replace(r"\&companyname[0-9]\&", "companyname", regex=True)

df[대체어_위치_표준어]

,방언,표준어,연령대,출처 파일
69,아니 내가 제일 젊진 않고 여기 저 저 name이 엄마 name이 엄마가 더 젊지.,아니 내가 제일 젊진 않고 여기 저 저 name이 엄마 name이 엄마가 더 젊지.,60대 이상,DGDQ20000020.json
70,name이 엄마가 나보다 한 살 아래지 나이로 따지면,name이 엄마가 나보다 한 살 아래지 나이로 따지면,60대 이상,DGDQ20000020.json
288,강릉시 주문진읍에 사는 스무살 name이라고 합니다.,강릉시 주문진읍에 사는 스무살 name이라고 합니다.,20대,DGDQ20000027.json
487,저희가 name 선생님이라는 캡틴을 잘 만나 가지고,저희가 name 선생님이라는 캡틴을 잘 만나 가지고,20대,DGDQ20000027.json
609,name이라는 애가 있습니다.,name이라는 애가 있습니다.,20대,DGDQ20000027.json
...,...,...,...,...
1063038,그러다 name가 저보고 너 다 들었지?,그러다 name가 저보고 너 다 들었지?,50대,DGIN21920687.json
1063044,name보다 행복한 아인 거 같아요.,name보다 행복한 아인 거 같아요.,50대,DGIN21920687.json
1063045,name는 엄마랑 아빠가 다같이 못산다 하는데,name는 엄마랑 아빠가 다같이 못산다 하는데,50대,DGIN21920687.json
1063047,그래 사램은 말이다 name아.,그래 사람은 말이다 name아.,50대,DGIN21920687.json


### 데이터 정제

In [11]:
df["길이"] = [min(len(dia.split()), len(stan.split())) for dia, stan in zip(df["방언"], df["표준어"])]
df["길이"]

0          1
1          1
2          3
3          4
4          1
          ..
1573232    3
1573233    3
1573234    2
1573235    3
1573236    2
Name: 길이, Length: 1573237, dtype: int64

In [12]:
suf_len_df = df[df["길이"] > 4]
len(suf_len_df)

841540

In [13]:
dia_sur_len = len(suf_len_df[suf_len_df["방언"] != suf_len_df["표준어"]])
print("길이로 제외한 후 남은 방언 샘플 수: {0}".format(dia_sur_len))

길이로 제외한 후 남은 방언 샘플 수: 557969


방언 데이터의 순도가 전라 방언에 비해 훨씬 높다고 봐야할 것 같다.

In [14]:
suf_len_df.to_csv("../datas/output/gangwon_dialect_data_age_processed.csv")

In [16]:
len(suf_len_df)

841540

### 데이터 샘플링

In [17]:
dial_df = suf_len_df[suf_len_df["방언"] != suf_len_df["표준어"]]
stan_df = suf_len_df[suf_len_df["방언"] == suf_len_df["표준어"]]

In [18]:
len(stan_df)

283571

강원 방언는 표준어 샘플이 방언 샘플보다 훨씬 작으니 훈련 셋을 그냥 사용하도록 하겠다.

In [19]:
sampled_df = suf_len_df

In [20]:
sampled_df.to_csv("../datas/output/gangwon_dialect_data_age_sampled.csv")

### 형태소 토큰화

In [21]:
import MeCab

def pos(sentence: str):
    """한국어 토큰을 분리합니다. 토큰과 품사를 튜플 리스트로 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list[tuple]): 토큰과 품사 리스트
    """
    t = MeCab.Tagger()
    tag_result = t.parse(sentence)
    tag_result = tag_result.replace("\t", ".@!").replace("\n", ".@!").split(".@!")
    tag_word = tag_result[::2][:-1] # 마지막 EOS는 자른다
    tag_info = tag_result[1::2][:-1] # 마지막 EOS는 자른다
    return [(word, info.split(',')[0]) for word, info in zip(tag_word, tag_info)]

def morphs(sentence: str):
    """한국어 토큰을 분리합니다. 토큰의 리스트를 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list): 토큰 리스트
    """
    t = MeCab.Tagger()
    tag_result = t.parse(sentence)
    tag_result = tag_result.replace("\t", ".@!").replace("\n", ".@!").split(".@!")
    return tag_result[::2][:-1]

def space_tokenize(original_sentence, token_list):
    """
    문장을 decode할 때 한국어의 띄어쓰기도 되살릴 수 있도록 token에 이를 반영하도록 처리하는 함수
    """
    
    space_token = original_sentence.split()

    if len(space_token) == 0:
        return token_list

    token_idx = 0
    cur_token = space_token[token_idx]
    cum_word = ""
    for i, word in enumerate(token_list):
        cum_word += word

        if len(cum_word) > len(word):
            token_list[i] = "##" + token_list[i]
            
        if cum_word == cur_token:
            token_idx += 1
            cum_word = ""
            cur_token = space_token[min(token_idx, len(space_token) - 1)]
    
    return token_list

def morph_and_preprocess(sentence: str):
    """한국어 토큰을 분리하고 전처리합니다. 토큰의 리스트를 반환합니다.
        
        매개변수:
            sentence (str): 토큰화할 문장 
        반환값:
            token list (list): 토큰 리스트
    """
    
    pos_result = pos(sentence)
    word_list = [word for word, _ in pos_result]
    word_list = space_tokenize(sentence, word_list) # 공백 복원 토큰화
    
    return word_list

In [22]:
dialect_list = sampled_df["방언"].to_list()
dialect_list = list(map(morph_and_preprocess, dialect_list))

standard_list = sampled_df["표준어"].to_list()
standard_list = list(map(morph_and_preprocess, standard_list))

In [23]:
list_to_spaced_sentence = lambda li : " ".join(li)
sampled_df["방언"] = list(map(list_to_spaced_sentence, dialect_list))
sampled_df["표준어"] = list(map(list_to_spaced_sentence, standard_list))

sampled_df.head()

C:\Users\wnsgh\AppData\Local\Temp\ipykernel_11064\2846202428.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df["방언"] = list(map(list_to_spaced_sentence, dialect_list))
C:\Users\wnsgh\AppData\Local\Temp\ipykernel_11064\2846202428.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_df["표준어"] = list(map(list_to_spaced_sentence, standard_list))


,방언,표준어,연령대,출처 파일,길이
6,바다 ##의 무슨 얘기 ##를 해야 되 ##나 ##?,바다 ##의 무슨 얘기 ##를 해야 되 ##나 ##?,60대 이상,DGDQ20000020.json,5
10,옛날 ##에 바다 ##에서 내 ##가 잘 ##한다고 엠 ##비씨 ##까지 갔 ##다...,옛날 ##에 바다 ##에서 내 ##가 잘 ##한다고 엠 ##비씨 ##까지 갔 ##다...,60대 이상,DGDQ20000020.json,8
13,구 ##십 몇 년 도 ##에 했 ##는데요 ##?,구 ##십 몇 년 도 ##에 했 ##는데요 ##?,60대 이상,DGDQ20000020.json,5
14,그때 ##가 엠 ##비씨 방송국 ##에서 해녀 하 ##라고 또 인터뷰 하 ##자 하...,그때 ##가 엠 ##비씨 방송국 ##에서 해녀 하 ##라고 또 인터뷰 하 ##자 하...,60대 이상,DGDQ20000020.json,9
15,그래서 엠 ##비씨 방송 공개 홀 강릉 가 가지 ##고 노래 ##도 부르 ##구,그래서 엠 ##비씨 방송 공개 홀 강릉 가 가지 ##고 노래 ##도 부르 ##구,60대 이상,DGDQ20000020.json,10


In [24]:
sampled_df.to_csv("../datas/output/gangwon_dialect_data_형태소.csv")